### Continuous Unsupervised Adaptation

In [1]:
# import modules and libraries
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable

# functions to create datasets and models
from data.data import *
from models.models import *

In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available')
else:
    print('CUDA is available')
    device = torch.device('cuda')

CUDA is available


### Parameters

In [3]:
"""Parameters for this experiment"""
# params for loading the data
path = 'data/Paderborn_FD/'
source_domain = 'a'
target_domains = ['b', 'c', 'd']

# params for configuring the model
input_dim = 1
hidden_dim = 256
output_dim = 3
drop_prob = 0.5

# params for training the model
epochs_pre = 50 # pre-training
epochs = 50

# optimizing the model
lr = 1e-5
d_lr = 1e-5
batch_size = 20
beta1 = 0.5
beta2 = 0.9
lambda_rpy = 0.025

### Data Loading

In [4]:
# source domain data for training, validating and testing
train_dataloader_src, val_dataloader_src, test_dataloader_src = generate_dataloaders(path, source_domain, batch_size)

# target domain data for validating and testing
train_dataloader_tgt, val_dataloader_tgt, test_dataloader_tgt = [], [], []
for target_domain in target_domains:
    train_dataloader_temp, val_dataloader_temp, test_dataloader_temp = generate_dataloaders(path, target_domain, batch_size)
    
    train_dataloader_tgt.append(train_dataloader_temp)
    val_dataloader_tgt.append(val_dataloader_temp)
    test_dataloader_tgt.append(test_dataloader_temp)

# replay data for CDA
replay_dataset = ReplayDataset(train_dataloader_src.dataset)
replay_dataloader = DataLoader(dataset=replay_dataset, batch_size=batch_size)

### Model

In [5]:
encoder_src = Encoder_AMDA(
    input_dim=input_dim, # 1D input (sensor readings)
    hidden_dim=hidden_dim # hidden layers 64
)

classifier = Classifier_AMDA(
    hidden_dim=hidden_dim, # hidden layers 64
    output_dim=output_dim, # 3 classes (healthy, inner- and outer-bearing damages)
    dropout=drop_prob # dropout prob 0.5
)

encoder_tgt = Encoder_AMDA(
    input_dim=input_dim, # 1D input (sensor readings)
    hidden_dim=hidden_dim, # hidden layers 64
)

discriminator = Discriminator(
    hidden_dim=hidden_dim
)

if train_on_gpu:
    encoder_src = encoder_src.to(device)
    classifier = classifier.to(device)
    encoder_tgt = encoder_tgt.to(device)
    discriminator = discriminator.to(device)

#### Functions for training and testing the model

In [6]:
# functions for training source encoder and shared classifier
def train_src(encoder, classifier):
    """Train the source encoder and shared classifier on source domain"""
    encoder.train()
    classifier.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(
        list(encoder.parameters()) + list(classifier.parameters()),
        lr=lr,
        betas=(beta1, beta2)
    )

    for epoch in range(epochs_pre):
        accuracies = []
        losses = []

        for batch_idx, (x_src, y_src) in enumerate(train_dataloader_src):
            if train_on_gpu:
                x_src, y_src = x_src.to(device), y_src.to(device)

            optimizer.zero_grad()

            e_src = encoder(x_src)
            pred_src = classifier(e_src)
            loss_src = criterion(pred_src, y_src)
            loss_src.backward()

            optimizer.step()

            losses.append(loss_src.detach().item()) # detach the loss from compute graph 
            accuracies.append(y_src.eq(pred_src.detach().argmax(dim=1)).float().mean())
        
        # print for each epoch
        print(f'Epoch: {epoch + 1} \n \t Train Loss:{torch.tensor(losses).mean():0.2f} \t Train Acc:{torch.tensor(accuracies).mean():0.2f}')

        # for each epoch, evaluate your model on the validation data
        val_loss, val_acc = evaluate_src(encoder, classifier, phase = 'Val')
        print(f'\t Val Loss:{val_loss:0.2f} \t\t Val Acc:{val_acc:0.2f}')

    return encoder, classifier

# testing model on source domain
def evaluate_src(encoder, classifier, phase='Val'): # phase can be validate or test
    """Evaluate the trained network on source domain"""
    encoder.eval()
    classifier.eval()

    criterion = nn.CrossEntropyLoss()
    
    losses = []
    accuracies = []
    dataloader = val_dataloader_src if phase == 'Val' else test_dataloader_src
    
    with torch.no_grad():
        for batch_idx, (x_src, y_src) in enumerate(dataloader):
            if train_on_gpu:
                x_src, y_src = x_src.to(device), y_src.to(device)

            e_src = encoder(x_src)
            pred_src = classifier(e_src)
            loss_src = criterion(pred_src, y_src)
            
            # append loss for each batch
            losses.append(loss_src.detach().item()) # detach the loss from compute graph 
            accuracies.append(y_src.eq(pred_src.detach().argmax(dim=1)).float().mean())
            
        mean_loss = torch.tensor(losses).mean()
        mean_acc = torch.tensor(accuracies).mean()
        return mean_loss, mean_acc

In [7]:
# Continuous Unsupervised Adaptation (CUA)
# functions for training target encoder
def train_tgt(encoder_src, encoder_tgt, classifier, discriminator, phase):
    """Train the target encoder on target domains with source encoder and test the network"""
    encoder_tgt.train()
    discriminator.train()
    classifier.eval()

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(
        encoder_tgt.parameters(),
        lr=lr,
        betas=(beta1, beta2)
    )
    d_optimizer = Adam(
        discriminator.parameters(),
        lr=d_lr,
        betas=(beta1, beta2)
    )

    for epoch in range(epochs):
        losses = []
        losses_d = []
        losses_tgt = []
        losses_rpy = []
        accuracies = []

        for batch_idx, ((x_src, y_src), (x_tgt, y_tgt), (x_rpy, y_rpy)) in enumerate(zip(train_dataloader_src, train_dataloader_tgt[phase], replay_dataloader)):
            if(train_on_gpu):
                x_src, y_src = x_src.to(device), y_src.to(device)
                x_tgt, y_tgt = x_tgt.to(device), y_tgt.to(device)
                x_rpy, y_rpy = x_rpy.to(device), y_rpy.to(device)

            optimizer.zero_grad()
            d_optimizer.zero_grad()

            # part 1: sequential unsupervised adaptation
            # learn e_src and _tgt: source and target mapping represenations
            e_src = encoder_src(x_src)
            e_tgt = encoder_tgt(x_tgt)
            e_concat = torch.concat((e_src.detach(), e_tgt.detach()), 0)
            # minimize distance between source and target empirical mapping distribution using adversarial discriminator
            d_concat = discriminator(e_concat)

            label_src = Variable(torch.ones(e_src.size(0)).long()).to(device)
            label_tgt = Variable(torch.zeros(e_tgt.size(0)).long()).to(device)
            label_concat = torch.cat((label_src, label_tgt), 0)

            loss_d = criterion(d_concat, label_concat)
            loss_d.backward()
            d_optimizer.step()

            e_tgt = encoder_tgt(x_tgt)
            pred_tgt = classifier(e_tgt)
            d_tgt = discriminator(e_tgt)
            label_tgt = Variable(torch.ones(e_tgt.size(0)).long()).to(device)
            loss_tgt = criterion(d_tgt, label_tgt)

            # part 2: continuous replay adaptation
            # additional replay loss to retain 'prior knowledge'
            pred_rpy = classifier(encoder_tgt(x_rpy))
            loss_rpy = criterion(pred_rpy, y_rpy)
            loss = loss_tgt + lambda_rpy * loss_rpy
            loss.backward()

            optimizer.step()

            # training (with replay) losses and accuracies
            losses.append(loss.detach().item()) # detach the loss from compute graph
            losses_d.append(loss_d.detach().item())
            losses_tgt.append(loss_tgt.detach().item())
            losses_rpy.append(loss_rpy.detach().item())
            accuracies.append(y_tgt.eq(pred_tgt.detach().argmax(dim=1)).float().mean())
        
        # print for each epoch
        # print for each epoch
        print(f'Epoch: {epoch + 1} \n \t Train Loss:{torch.tensor(losses).mean():0.2f} \t Train Acc:{torch.tensor(accuracies).mean():0.2f}')
        print(f'\t Discriminator Loss:{torch.tensor(losses_d).mean():0.2f} \t Train (Replay) Loss:{torch.tensor(losses_rpy).mean():0.2f} \t Train (Adversarial) Loss:{torch.tensor(losses_tgt).mean():0.2f}')
        
        # for each epoch, evaluate your model on the validation data
        val_loss, val_acc = evaluate_tgt(encoder_tgt, classifier, val_dataloader_tgt[phase])
        print(f'\t Val (with Target) Loss:{val_loss:0.2f} \t Val (with Target) Acc:{val_acc:0.2f}')

    return encoder_tgt
    
# testing model on target domains
def evaluate_tgt(encoder, classifier, dataloader):
    """Evaluate the network on current target domain"""
    encoder.eval()
    classifier.eval()

    criterion = nn.CrossEntropyLoss()
    
    losses = []
    accuracies = []
    
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(dataloader): # target dataloader
            if(train_on_gpu):
                x, y = x.to(device), y.to(device)

            pred_tgt = classifier(encoder(x))
            loss_tgt = criterion(pred_tgt, y)
            
            # append loss for each batch
            losses.append(loss_tgt.detach().item()) # detach the loss from compute graph 
            accuracies.append(y.eq(pred_tgt.detach().argmax(dim=1)).float().mean())
            
        mean_loss = torch.tensor(losses).mean()
        mean_acc = torch.tensor(accuracies).mean()
        return mean_loss, mean_acc

In [8]:
# subsample previously seen examples and their respective classification scores as 'soft labels' from the target domains
def generate_data_tuple(encoder_tgt, classifier, dataloader):
    print('===> generate new replay dataset!')
    encoder_tgt.eval()
    classifier.eval()

    all_x, all_pred = [], []
    for batch_idx, (x_tgt, y_tgt) in enumerate(dataloader):
        if(train_on_gpu):
            x_tgt, y_tgt = x_tgt.to(device), y_tgt.to(device)

        # perform classification with the target mapping representations
        pred_tgt = classifier(encoder_tgt(x_tgt))

        all_x.append(to_np(x_tgt))
        all_pred.append(to_np(pred_tgt.detach().argmax(dim=1)))
        
    x, pred = np.concatenate(all_x, 0), np.concatenate(all_pred, 0)
    print('generated: ', x.shape, pred.shape)
    return x, pred

### Pre-training

In [9]:
# pre-training
encoder_src, classifier = train_src(encoder_src, classifier)

Epoch: 1 
 	 Train Loss:1.06 	 Train Acc:0.46
	 Val Loss:1.03 		 Val Acc:0.45
Epoch: 2 
 	 Train Loss:1.01 	 Train Acc:0.46
	 Val Loss:0.98 		 Val Acc:0.45
Epoch: 3 
 	 Train Loss:0.95 	 Train Acc:0.46
	 Val Loss:0.93 		 Val Acc:0.45
Epoch: 4 
 	 Train Loss:0.91 	 Train Acc:0.44
	 Val Loss:0.89 		 Val Acc:0.45
Epoch: 5 
 	 Train Loss:0.85 	 Train Acc:0.57
	 Val Loss:0.79 		 Val Acc:0.60
Epoch: 6 
 	 Train Loss:0.73 	 Train Acc:0.69
	 Val Loss:0.66 		 Val Acc:0.75
Epoch: 7 
 	 Train Loss:0.62 	 Train Acc:0.75
	 Val Loss:0.58 		 Val Acc:0.76
Epoch: 8 
 	 Train Loss:0.56 	 Train Acc:0.77
	 Val Loss:0.53 		 Val Acc:0.78
Epoch: 9 
 	 Train Loss:0.52 	 Train Acc:0.78
	 Val Loss:0.50 		 Val Acc:0.79
Epoch: 10 
 	 Train Loss:0.49 	 Train Acc:0.79
	 Val Loss:0.47 		 Val Acc:0.80
Epoch: 11 
 	 Train Loss:0.47 	 Train Acc:0.80
	 Val Loss:0.46 		 Val Acc:0.80
Epoch: 12 
 	 Train Loss:0.46 	 Train Acc:0.80
	 Val Loss:0.44 		 Val Acc:0.81
Epoch: 13 
 	 Train Loss:0.45 	 Train Acc:0.81
	 Val Loss:0.4

In [10]:
encoder_tgt.load_state_dict(encoder_src.state_dict())

# adaptation
for phase in range(len(target_domains)):
    if(phase == 0): # Domain A to B
        print(f'\nPhase {phase + 1} (Domain {source_domain} to Domain {target_domains[phase]})')
    else: # Domain B to C, and Domain C to D
        print(f'\nPhase {phase + 1} (Domain {target_domains[phase - 1]} to Domain {target_domains[phase]})')
    
    encoder_tgt = train_tgt(encoder_src, encoder_tgt, classifier, discriminator, phase)
    
    print('\nTesting Accuracy on Previously Seen and Current Domains:')
    print(f'Source Domain {source_domain}:')
    src_test_loss, src_test_acc = evaluate_src(encoder_tgt, classifier, phase='Test')
    print(f'\t Test (with Source) Loss:{src_test_loss:0.2f} \t Test (with Source) Acc:{src_test_acc:0.2f}') 
    for p in range(phase + 1):
        print(f'Target Domain {target_domains[p]}:')
        tgt_test_loss, tgt_test_acc = evaluate_tgt(encoder_tgt, classifier, test_dataloader_tgt[phase])
        print(f'\t Test (with Target) Loss:{tgt_test_loss:0.2f} \t Test (with Target) Acc:{tgt_test_acc:0.2f}')

    # update replay data and dataloader
    data_tuple = generate_data_tuple(encoder_tgt, classifier, train_dataloader_tgt[phase])
    replay_dataset.replay_dataset.update(data_tuple)
    print(replay_dataset.__len__())
    replay_dataloader = DataLoader(dataset=replay_dataset, batch_size=batch_size)


Phase 1 (Domain a to Domain b)
Epoch: 1 
 	 Train Loss:1.11 	 Train Acc:0.53
	 Discriminator Loss:0.68 	 Train (Replay) Loss:3.27 	 Train (Adversarial) Loss:1.03
	 Val (with Target) Loss:2.38 	 Val (with Target) Acc:0.75
Epoch: 2 
 	 Train Loss:0.81 	 Train Acc:0.69
	 Discriminator Loss:0.67 	 Train (Replay) Loss:4.65 	 Train (Adversarial) Loss:0.70
	 Val (with Target) Loss:1.03 	 Val (with Target) Acc:0.78
Epoch: 3 
 	 Train Loss:0.78 	 Train Acc:0.70
	 Discriminator Loss:0.67 	 Train (Replay) Loss:3.34 	 Train (Adversarial) Loss:0.70
	 Val (with Target) Loss:2.70 	 Val (with Target) Acc:0.59
Epoch: 4 
 	 Train Loss:0.75 	 Train Acc:0.63
	 Discriminator Loss:0.67 	 Train (Replay) Loss:2.25 	 Train (Adversarial) Loss:0.70
	 Val (with Target) Loss:2.24 	 Val (with Target) Acc:0.59
Epoch: 5 
 	 Train Loss:0.73 	 Train Acc:0.67
	 Discriminator Loss:0.66 	 Train (Replay) Loss:1.99 	 Train (Adversarial) Loss:0.68
	 Val (with Target) Loss:0.71 	 Val (with Target) Acc:0.82
Epoch: 6 
 	 Train

In [11]:
src_test_loss, src_test_acc = evaluate_src(encoder_tgt, classifier, phase='Test')
print(f'\t Test (with Source) Loss:{src_test_loss:0.2f} \t Test (with Source) Acc:{src_test_acc:0.2f}') 
for phase in range(len(target_domains)):
    if(phase == 0): # Domain A to B
        print(f'\nPhase {phase + 1} (Domain {source_domain} to Domain {target_domains[phase]})')
    else: # Domain B to C, and Domain C to D
        print(f'\nPhase {phase + 1} (Domain {target_domains[phase - 1]} to Domain {target_domains[phase]})')
    
    tgt_test_loss, tgt_test_acc = evaluate_tgt(encoder_tgt, classifier, test_dataloader_tgt[phase])
    print(f'\t Test (with Target) Loss:{tgt_test_loss:0.2f} \t Test (with Target) Acc:{tgt_test_acc:0.2f}') 

	 Test (with Source) Loss:0.56 	 Test (with Source) Acc:0.75

Phase 1 (Domain a to Domain b)
	 Test (with Target) Loss:0.60 	 Test (with Target) Acc:0.79

Phase 2 (Domain b to Domain c)
	 Test (with Target) Loss:0.60 	 Test (with Target) Acc:0.82

Phase 3 (Domain c to Domain d)
	 Test (with Target) Loss:0.46 	 Test (with Target) Acc:0.82
